In [ ]:
import numpy as np
import pandas as pd
df_fraud=pd.read_csv('fraud_clean.csv')
df_ip=pd.read_csv('ip_clean.csv')

In [ ]:
df_ip['lower_bound_ip_address']=df_ip['lower_bound_ip_address'].astype(int)
df_fraud['ip_address']=df_fraud['ip_address'].astype(int)

In [ ]:
# Merge the datasets based on IP address range
df_merged = pd.DataFrame()

for index, row in df_ip.iterrows():
    lower_bound = row['lower_bound_ip_address']
    upper_bound = row['upper_bound_ip_address']
    country = row['country']

    # Filter fraud data within the IP range
    filtered_data = df_fraud[(df_fraud['ip_address'] >= lower_bound) & (df_fraud['ip_address'] <= upper_bound)]

    # Add country column to filtered data
    filtered_data['country'] = country

    # Append filtered data to df_merged
    df_merged = pd.concat([df_merged, filtered_data], ignore_index=True) # Changed 'ignoreAC_index' to 'ignore_index'

# Drop unnecessary columns
#df_merged = df_merged.drop(['ip_address'], axis=1)


Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['country'] = country
<ipython-input-13-3a84367012c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['country'] = country
<ipython-input-13-3a84367012c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['country'] = country
<ipython-input-13-3a84367012c9>:13: SettingWithCopy

In [ ]:
df_merged.to_csv('ip_fraud_merged.csv')


In [ ]:
df_merged.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,class,country
0,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,0.0,China
1,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,0.0,Thailand
2,379446,2015-04-11 13:13:27,2015-07-03 22:27:10,49,FAWCZLEWWUDVD,SEO,Chrome,M,24,0.0,Japan
3,236894,2015-06-16 04:02:41,2015-07-25 01:29:33,22,YXWLWLZBQVBMG,Ads,Chrome,M,31,0.0,Japan
4,53226,2015-05-20 08:29:13,2015-06-16 06:49:46,9,ATCHXLVPDRFJH,SEO,Chrome,F,35,0.0,India


In [22]:
import pandas as pd

# Load the data
df_fraud = pd.read_csv('ip_fraud_merged.csv')

# Convert 'purchase_time' to datetime format
df_fraud['purchase_time'] = pd.to_datetime(df_fraud['purchase_time'], errors='coerce')

# Drop rows with NaT in 'purchase_time'
df_fraud = df_fraud.dropna(subset=['purchase_time'])

# Extract hour of the day
df_fraud['hour_of_day'] = df_fraud['purchase_time'].dt.hour

# Extract day of the week
df_fraud['day_of_week'] = df_fraud['purchase_time'].dt.dayofweek

# Calculate transaction frequency per user per day
df_fraud['transaction_date'] = df_fraud['purchase_time'].dt.date
transaction_frequency = df_fraud.groupby(['user_id', 'transaction_date']).size().reset_index(name='transaction_count_per_day')

# Merge transaction frequency back to the original dataframe
df_fraud = df_fraud.merge(transaction_frequency, on=['user_id', 'transaction_date'], how='left')

# Calculate transaction velocity (rolling count of transactions in the past hour)
df_fraud = df_fraud.sort_values(by=['user_id', 'purchase_time'])

# Initialize a new column for transaction velocity
df_fraud['transaction_velocity_past_hour'] = 0

# Calculate the rolling count of transactions in the past hour for each user
for user_id in df_fraud['user_id'].unique():
    user_data = df_fraud[df_fraud['user_id'] == user_id].set_index('purchase_time').sort_index()
    rolling_count = user_data.rolling('1H').count() - 1
    df_fraud.loc[df_fraud['user_id'] == user_id, 'transaction_velocity_past_hour'] = rolling_count['user_id'].values

# Drop the intermediate 'transaction_date' column
df_fraud.drop(columns=['transaction_date'], inplace=True)

# Display the dataframe with new features
#


In [23]:
df_fraud.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,hour_of_day,day_of_week,transaction_count_per_day,transaction_velocity_past_hour
30215,30215,2,2015-01-11 03:47:13,2015-02-21 10:03:37,54,FGBQNDNBETFJJ,SEO,Chrome,F,25,880217484,0,United States,10,5,1,0
95232,95232,4,2015-06-02 16:40:57,2015-09-26 21:32:16,41,MKFUIVOHLJBYN,Direct,Safari,F,38,2785906106,0,Switzerland,21,5,1,0
11691,11691,8,2015-05-28 07:53:06,2015-08-13 11:53:07,47,SCQGQALXBUQZJ,SEO,Chrome,M,25,356056736,0,United States,11,3,1,0
101958,101958,12,2015-01-10 06:25:12,2015-03-04 20:56:37,35,MSNWCFEHKTIOY,Ads,Safari,M,19,2985180352,0,Mexico,20,2,1,0
19306,19306,16,2015-02-03 13:48:23,2015-03-12 12:46:23,9,FROZWSSWOHZBE,Direct,IE,M,32,578312545,0,United States,12,3,1,0


In [24]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Define columns to be scaled (numerical columns)
numerical_cols = ['transaction_count_per_day', 'transaction_velocity_past_hour', 'age', 'purchase_value']

# Apply the scaler to the numerical columns
df_fraud[numerical_cols] = scaler.fit_transform(df_fraud[numerical_cols])

# Display the scaled dataframe
print(df_fraud.head())


        Unnamed: 0  user_id          signup_time       purchase_time  \
30215        30215        2  2015-01-11 03:47:13 2015-02-21 10:03:37   
95232        95232        4  2015-06-02 16:40:57 2015-09-26 21:32:16   
11691        11691        8  2015-05-28 07:53:06 2015-08-13 11:53:07   
101958      101958       12  2015-01-10 06:25:12 2015-03-04 20:56:37   
19306        19306       16  2015-02-03 13:48:23 2015-03-12 12:46:23   

        purchase_value      device_id  source browser sex       age  \
30215         0.310345  FGBQNDNBETFJJ     SEO  Chrome   F  0.120690   
95232         0.220690  MKFUIVOHLJBYN  Direct  Safari   F  0.344828   
11691         0.262069  SCQGQALXBUQZJ     SEO  Chrome   M  0.120690   
101958        0.179310  MSNWCFEHKTIOY     Ads  Safari   M  0.017241   
19306         0.000000  FROZWSSWOHZBE  Direct      IE   M  0.241379   

        ip_address  class        country  hour_of_day  day_of_week  \
30215    880217484      0  United States           10            5   


In [25]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Select categorical columns to encode
categorical_cols = ['source', 'browser', 'sex', 'country']

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Apply the OneHotEncoder to categorical columns
encoded_cats = pd.DataFrame(onehot_encoder.fit_transform(df_fraud[categorical_cols]))

# Re-attach column names
encoded_cats.columns = onehot_encoder.get_feature_names_out(categorical_cols)

# Drop original categorical columns
df_fraud = df_fraud.drop(columns=categorical_cols)

# Concatenate the encoded columns with the original dataframe
df_fraud = pd.concat([df_fraud.reset_index(drop=True), encoded_cats.reset_index(drop=True)], axis=1)

# Display the dataframe with new features
df_fraud.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,hour_of_day,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,30215,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,880217484,0,10,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,95232,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,2785906106,0,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11691,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,356056736,0,11,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101958,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,2985180352,0,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19306,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,578312545,0,12,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_fraud = df_fraud.loc[:, ~df_fraud.columns.str.contains('^Unnamed')]


In [27]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,hour_of_day,day_of_week,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,880217484,0,10,5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,2785906106,0,21,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,356056736,0,11,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,2985180352,0,20,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,578312545,0,12,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
df_fraud.to_csv('fraud_preprocessed.csv', index=False)
